In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("WakeCounty_original_data.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,adopted
1,2,1,operating
2,3,1,budget
3,4,1,and
4,5,1,capital


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,adopted
1,1,operating
2,1,budget
3,1,and
4,1,capital
5,1,improvement
6,1,program
7,1,for
8,1,the
9,1,fiscal


In [5]:
data.describe()

,Unnamed: 0,page_number
count,122016.00000,122016.000000
mean,61008.50000,246.660463
std,35223.12956,144.223414
min,1.00000,1.000000
25%,30504.75000,113.000000
50%,61008.50000,242.000000
75%,91512.25000,375.000000
max,122016.00000,498.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['165",2,"sources']
['336",3,"was']
['806",9,"management']
['1071",11,"residents']
['1333",12,"priorities']
['1407",12,"for']
['1438",12,"rate']
['1482",12,"our']
['1524",12,"or']
['1539",12,"three']
['1606",12,"the']
['1746",13,"for']
['2014",14,"million']
['2056",14,"non']
['2174",15,"assumes']
['2243",15,"included']
['2305",15,"the']
['2326",15,"tax']
['2353",15,"because']
['2373",15,"sales']
['2680",16,"offers']
['2728",16,"available']
['2802",16,"changes']
['2813",16,"to']
['2854",16,"lower']
['2883",16,"budget']
['3030",17,"lation']
['3353",18,"the']
['3375",18,"schools']
['3442",18,"for']
['3464",18,"board']
['3504",18,"and']
['3593",18,"schools']
['3953",19,"court']
['4011",19,"and']
['4259",20,"replacement']
['4421",20,"that']
['4426",20,"the']
['4537",21,"general']
['4542",21,"needs']
['4557",21,"is']
['4559",21,"more']
['4652",21,"public']
['4692",21,"partially']
['4780",21,"debt']
['4818",21,"facilities']
['5020",22,"costs']
['5295",23,"staffing']
['5392",23,"of']
['5398",2

['35427",163,"tornado']
['35441",163,"public']
['35681",164,"been']
['35823",164,"with']
['35841",164,"totaled']
['36069",165,"the']
['36232",166,"hours']
['36388",169,"provision']
['36482",169,"resources']
['36510",169,"at']
['36590",170,"emergency']
['36704",170,"human']
['36760",170,"this']
['36769",170,"in']
['36775",170,"personnel']
['36975",170,"is']
['36976",170,"dependent']
['36991",170,"everincreasing']
['36997",170,"significant']
['37023",170,"this']
['37190",171,"collection']
['37468",173,"and']
['37576",173,"of']
['38042",174,"homicides']
['38098",175,"positively']
['38289",175,"to']
['38368",175,"however']
['38468",176,"city']
['38469",176,"of']
['38495",176,"to']
['38659",177,"gm']
['38793",177,"projected']
['38860",178,"standards']
['39053",179,"january']
['39551",181,"regulations']
['39581",181,"guilford']
['39760",181,"code']
['39873",182,"building']
['40085",182,"in']
['40093",182,"builders']
['40313",184,"fy']
['40376",184,"residential']
['40471",185,"complaintdriven

['71727",329,"gibson']
['71768",329,"interior']
['71783",329,"st']
['72120",331,"other']
['72134",331,"point']
['72377",333,"roofing']
['72493",334,"roofing']
['72494",334,"building']
['72790",335,"paving']
['72835",336,"building']
['72983",337,"park']
['73220",339,"originally']
['73229",339,"tyler']
['73253",339,"screen']
['73312",339,"social']
['73360",340,"body']
['73377",340,"cameras']
['73487",341,"replace']
['73539",341,"public']
['73609",343,"two']
['73640",343,"capital']
['73716",343,"inform']
['73778",343,"medical']
['73820",343,"over']
['73852",343,"assessment']
['74116",344,"commissioners']
['74205",344,"the']
['74224",344,"a']
['74355",344,"county']
['74367",344,"proposed']
['74422",344,"budget']
['74636",346,"includes']
['74648",346,"and']
['74719",346,"to']
['74786",346,"administration']
['74978",347,"and']
['75025",347,"debt']
['75121",348,"on']
['75190",348,"admin']
['75280",348,"either']
['75407",349,"year']
['75473",349,"bonds']
['75504",349,"tdma']
['75661",351,"cons

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.006*"4652",21,"public" + 0.006*"53094",239,"and" + 0.006*"92076",476,"controlled" + 0.006*"53848",242,"personnel" + 0.006*"45028",204,"volunteers" + 0.006*"17232",79,"renovation" + 0.006*"76301",357,"this" + 0.006*"82024",390,"current" + 0.006*"49162",221,"departmental" + 0.006*"36510",169,"at"
Topic: 1 word: 0.006*"19014",88,"quality" + 0.006*"19047",88,"additional" + 0.006*"10326",41,"public" + 0.006*"90003",460,"conduct" + 0.006*"5020",22,"costs" + 0.006*"54160",244,"workforce" + 0.006*"11328",47,"fund" + 0.006*"48062",217,"school" + 0.006*"41481",188,"service" + 0.006*"84262",397,"high"
Topic: 2 word: 0.006*"15892",72,"change" + 0.006*"58034",258,"of" + 0.006*"53718",242,"schools" + 0.006*"27099",127,"program" + 0.006*"8941",36,"types" + 0.006*"90640",463,"technicians" + 0.006*"37190",171,"collection" + 0.006*"68681",311,"theft" + 0.006*"48325",218,"the" + 0.006*"52250",235,"includes"
Topic: 3 word: 0.006*"33551",155,"promotions" + 0.006*"51635",231,"aicp" + 0.006*

In [15]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\messi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
